In [1]:
import pandas as pd
import re
from datetime import datetime
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [48]:
try:
    conexion= psycopg2.connect(
        database= "Lab.14",
        user= "postgres",
        password= "admin",
        host= "localhost",
        port= "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

## Ejercicio 1. Queries Generales

1.1. Calcula el promedio de temperatura.



In [5]:
cursor= conexion.cursor()
query_promedio= """
SELECT AVG(temperatura_°c)
FROM tiempo t;
"""
cursor.execute(query_promedio)   
cursor.fetchone()

(Decimal('21.2498387720882239'),)

1.2. Obtén los municipios en los cuales coincidan las medias de la sensación térmica y de la temperatura. 



Lo comento porque no tengo esa variable y para que no me de error

In [ ]:
# cursor=conexion.cursor()
# query_coincidencia="""
# SELECT  m."nombre_municipio", AVG(temperatura_°c),  AVG(sensacion_termica)
# FROM tiempo t
# INNER JOIN municipios m ON m."id_municipio" = t."id_municipio"
# GROUP BY id_municipio
# HAVING AVG(temperatura_°c) = AVG(sensacion_termica)

# """

1.3. Obtén el local más cercano de cada municipio



In [49]:
cursor = conexion.cursor()
query_distancia=""" 
SELECT  m.nombre_municipio, categorias, MIN(l.distancia) AS distancia1
FROM categorias c
INNER JOIN lugar l ON l.id_categoria = c.id_categorias
INNER JOIN municipios m ON l.id_municipio = m.id_municipio
GROUP BY id_categorias, m.id_municipio

"""
cursor.execute(query_distancia)
df_query3 = pd.DataFrame(cursor.fetchall(), columns=["Municipio","Categorias","Distancia"])
df_query3.head()
#AHORA VUELVO  ESTE

,Municipio,Categorias,Distancia
0,Robledo de Chavela,Park,977.0
1,Villanueva de la Cañada,Park,577.0
2,Collado Villalba,Park,210.0
3,Coslada,Park,70.0
4,Navacerrada,Park,95.0


1.4. Localiza los municipios que posean algún localizador a una distancia mayor de 2000 y que posean al menos 25 locales.



In [50]:
cursor = conexion.cursor()
query_distancia=""" 
SELECT * FROM (
            SELECT  m.nombre_municipio AS nombre_mun, count(c.id_categorias) AS numero_locales, sum(distancia) AS total_distancia
                FROM categorias c
                INNER JOIN lugar l ON l.id_categoria = c.id_categorias
                INNER JOIN municipios m ON l.id_municipio = m.id_municipio
                GROUP BY m.id_municipio)
WHERE (total_distancia > 2000 AND numero_locales > 25);
"""
cursor.execute(query_distancia)
df_query4 = pd.DataFrame(cursor.fetchall(), columns=["Municipio","Numero_locales","Distancia"])
df_query4.head()

,Municipio,Numero_locales,Distancia
0,Majadahonda,27,28639.0
1,Torrejón de Ardoz,30,36768.0
2,Madrid,122,138753.0
3,Alcobendas,53,62547.0
4,Alcalá de Henares,52,49584.0


1.5. Teniendo en cuenta que la lluvia se considera leve con entre 6 y 20 mm, moderada con una media de entre 21 y 40 mm, fuerte con media de entre 41 y 70 mm y muy fuerte entre 71 y 120 mm. Calcula cuántas luvias de cada tipo tenemos en cada uno de los días. Este ejercicio debes solucionarlo con la sentencia CASE de SQL (no la hemos visto en clase, por lo que tendrás que buscar la documentación). 



In [51]:
cursor = conexion.cursor()
query_lluvia="""
SELECT sum(precipitacion_mm), fecha,
CASE
    WHEN sum(precipitacion_mm) < 6 THEN 'sin lluvia'
    WHEN sum(precipitacion_mm) >= 6 AND sum(precipitacion_mm) <= 20 THEN 'leve'
    WHEN sum(precipitacion_mm) >= 21 AND sum(precipitacion_mm) <= 40 THEN 'media'
    WHEN sum(precipitacion_mm) >= 41 AND sum(precipitacion_mm) <= 70 THEN 'fuerte'
    WHEN sum(precipitacion_mm) >= 71 AND sum(precipitacion_mm) <= 120 THEN 'muy fuerte'
    ELSE 'torrencial'
    END AS lluvia_range
FROM tiempo t
GROUP BY fecha;
"""
cursor.execute(query_lluvia)
df_query5 = pd.DataFrame(cursor.fetchall(), columns=["Precipitaciones","Fecha","Calificacion"])
df_query5.head()

,Precipitaciones,Fecha,Calificacion
0,3.2,2024-08-29 15:00:00,sin lluvia
1,34.8,2024-08-29 19:00:00,media
2,46.3,2024-08-29 17:00:00,fuerte
3,14.1,2024-08-29 16:00:00,leve
4,0.0,2024-08-31 03:00:00,sin lluvia


## Ejercicio 2. Vistas

2.1. Crea una vista que muestre la información de los locales que tengan incluido el código postal en su dirección. 



2.2. Crea una vista con los locales que tienen más de una categoría asociada.



2.3. Crea una vista que muestre el municipio con la temperatura más alta de cada día



2.4. Crea una vista con los municipios en los que haya una probabilidad de precipitación mayor del 100% durante mínimo 7 horas.



2.5. Obtén una lista con los parques de los municipios que tengan algún castillo.

## Ejercicio 2. Vistas

2.1. Crea una vista que muestre la información de los locales que tengan incluido el código postal en su dirección. 

2.2. Crea una vista con los locales que tienen más de una categoría asociada.

2.3. Crea una vista que muestre el municipio con la temperatura más alta de cada día

2.4. Crea una vista con los municipios en los que haya una probabilidad de precipitación mayor del 100% durante mínimo 7 horas.

2.5. Obtén una lista con los parques de los municipios que tengan algún castillo.

## Ejercicio 3. Tablas Temporales

3.1. Crea una tabla temporal que muestre cuántos días han pasado desde que se obtuvo la información de la tabla AEMET.

3.2. Crea una tabla temporal que muestre los locales que tienen más de una categoría asociada e indica el conteo de las mismas

3.3. Crea una tabla temporal que muestre los tipos de cielo para los cuales la probabilidad de precipitación mínima de los promedios de cada día es 5.

3.4. Crea una tabla temporal que muestre el tipo de cielo más y menos repetido por municipio.


## Ejercicio 4. SUBQUERIES

4.1. Necesitamos comprobar si hay algún municipio en el cual no tenga ningún local registrado.

4.2. Averigua si hay alguna fecha en la que el cielo se encuente "Muy nuboso con tormenta".

4.3. Encuentra los días en los que los avisos sean diferentes a "Sin riesgo".

4.4. Selecciona el municipio con mayor número de locales.

4.5. Obtén los municipios muya media de sensación térmica sea mayor que la media total.

4.6. Selecciona los municipios con más de dos fuentes.

4.7. Localiza la dirección de todos los estudios de cine que estén abiertod en el municipio de "Madrid".

4.8. Encuentra la máxima temperatura para cada tipo de cielo.

4.9. Muestra el número de locales por categoría que muy probablemente se encuentren abiertos.
